In [ ]:
import streamlit as st
import sqlite3
import datetime
import re
import os
from datetime import datetime, timedelta
import dateparser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
from langchain.chains import LLMChain

# Database setup
def init_db():
    conn = sqlite3.connect('appointments.db')
    c = conn.cursor()
    
    # Check if the table exists
    c.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='appointments'")
    table_exists = c.fetchone()
    
    if table_exists:
        # Check if email column exists
        try:
            c.execute("SELECT email FROM appointments LIMIT 1")
        except sqlite3.OperationalError:
            # Add email column if it doesn't exist
            st.write("Adding email column to existing database...")
            c.execute("ALTER TABLE appointments ADD COLUMN email TEXT DEFAULT 'no-email@example.com'")
            conn.commit()
    else:
        # Create new table with email column
        c.execute('''
        CREATE TABLE appointments
        (id INTEGER PRIMARY KEY AUTOINCREMENT,
         name TEXT NOT NULL,
         email TEXT NOT NULL,
         date TEXT NOT NULL,
         time TEXT NOT NULL,
         purpose TEXT,
         created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)
        ''')
        conn.commit()
    
    conn.close()

def add_appointment(name, email, date, time, purpose):
    conn = sqlite3.connect('appointments.db')
    c = conn.cursor()
    c.execute("INSERT INTO appointments (name, email, date, time, purpose) VALUES (?, ?, ?, ?, ?)",
              (name, email, date, time, purpose))
    conn.commit()
    conn.close()

def get_appointments(name=None, email=None, date=None):
    conn = sqlite3.connect('appointments.db')
    c = conn.cursor()
    
    query = "SELECT * FROM appointments"
    params = []
    
    if name and email and date:
        query += " WHERE name = ? AND email = ? AND date = ?"
        params = [name, email, date]
    elif name and email:
        query += " WHERE name = ? AND email = ?"
        params = [name, email]
    elif name and date:
        query += " WHERE name = ? AND date = ?"
        params = [name, date]
    elif email and date:
        query += " WHERE email = ? AND date = ?"
        params = [email, date]
    elif name:
        query += " WHERE name = ?"
        params = [name]
    elif email:
        query += " WHERE email = ?"
        params = [email]
    elif date:
        query += " WHERE date = ?"
        params = [date]
    
    query += " ORDER BY date, time"
    
    c.execute(query, params)
    appointments = c.fetchall()
    conn.close()
    return appointments

def check_appointment_exists(name, email, date, time):
    conn = sqlite3.connect('appointments.db')
    c = conn.cursor()
    
    try:
        c.execute("SELECT * FROM appointments WHERE name = ? AND email = ? AND date = ? AND time = ?", 
                (name, email, date, time))
        result = c.fetchone()
    except sqlite3.OperationalError:
        # Fallback if email column doesn't exist yet
        c.execute("SELECT * FROM appointments WHERE name = ? AND date = ? AND time = ?", 
                (name, date, time))
        result = c.fetchone()
    
    conn.close()
    return result is not None

# Setup the LLM
def setup_llm():
    # Get the API key from Streamlit secrets or environment variable
    api_key = os.environ.get('GEMINI_API_KEY') or st.secrets.get('GEMINI_API_KEY', '')
    
    if not api_key:
        st.error("Google API key not found. Please set it in your environment variables or Streamlit secrets.")
        st.stop()
    
    # Initialize the Google Generative AI model
    llm = ChatGoogleGenerativeAI(
        api_key=api_key,
        model="gemini-2.0-flash"
    )
    
    # Create a prompt template
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an appointment booking assistant. Your task is to:
        1. Help users book appointments by extracting name, email, date, time, and purpose.
        2. Help users retrieve their appointment information.
        3. Respond in a friendly and helpful manner.
        
        Important: Email address is required for all appointments. Always ask for it if not provided.
        
        When you identify appointment details, format your response with JSON-like tags:
        <APPOINTMENT_DETAILS>
        name: [extracted name]
        email: [extracted email]
        date: [extracted date in YYYY-MM-DD format]
        time: [extracted time in HH:MM format]
        purpose: [extracted purpose]
        action: [book/retrieve]
        </APPOINTMENT_DETAILS>
        
        If details are missing, indicate what information is needed but don't include the tags."""),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])
    
    # Create memory
    memory = ConversationBufferMemory(return_messages=True, input_key="input", memory_key="history")
    
    # Create chain
    chain = LLMChain(
        llm=llm,
        prompt=prompt,
        memory=memory
    )
    
    return chain

# Process the LLM response to extract appointment details
def extract_appointment_details(response_text):
    details_pattern = r'<APPOINTMENT_DETAILS>(.*?)</APPOINTMENT_DETAILS>'
    match = re.search(details_pattern, response_text, re.DOTALL)
    
    if not match:
        return None, response_text.strip()
    
    details_text = match.group(1).strip()
    details = {}
    
    # Extract each detail
    name_match = re.search(r'name:\s*(.*)', details_text)
    email_match = re.search(r'email:\s*(.*)', details_text)
    date_match = re.search(r'date:\s*(.*)', details_text)
    time_match = re.search(r'time:\s*(.*)', details_text)
    purpose_match = re.search(r'purpose:\s*(.*)', details_text)
    action_match = re.search(r'action:\s*(.*)', details_text)
    
    # Add extracted details to dictionary
    if name_match:
        details['name'] = name_match.group(1).strip()
    if email_match:
        details['email'] = email_match.group(1).strip()
    if date_match:
        date_str = date_match.group(1).strip()
        parsed_date = dateparser.parse(date_str)
        if parsed_date:
            details['date'] = parsed_date.strftime('%Y-%m-%d')
        else:
            details['date'] = date_str
    if time_match:
        time_str = time_match.group(1).strip()
        # Handle time parsing
        if ':' not in time_str:
            # Add a default :00 minutes if only hours are provided
            try:
                hour = int(time_str)
                time_str = f"{hour:02d}:00"
            except ValueError:
                pass
        details['time'] = time_str
    if purpose_match:
        details['purpose'] = purpose_match.group(1).strip()
    if action_match:
        details['action'] = action_match.group(1).strip()
    
    # Remove the appointment details section from the response
    clean_response = re.sub(details_pattern, '', response_text, flags=re.DOTALL).strip()
    
    return details, clean_response

# Helper function to validate email format
def is_valid_email(email):
    email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return bool(re.match(email_pattern, email))

# Chatbot logic
def process_message(user_input, llm_chain):
    try:
        # Get response from the LLM
        llm_response = llm_chain.invoke({"input": user_input})
        response_text = llm_response["text"]
        
        # Extract appointment details from the LLM response
        details, clean_response = extract_appointment_details(response_text)
        
        if not details:
            return clean_response
        
        # Process based on action
        if details.get('action') == 'book':
            # Check if we have all necessary details
            if not details.get('name'):
                details['name'] = st.session_state.get('current_name', '')
            if not details.get('email'):
                details['email'] = st.session_state.get('current_email', '')
            
            # Check for missing information
            missing = []
            if not details.get('name'):
                missing.append("name")
            if not details.get('email'):
                missing.append("email")
            elif not is_valid_email(details.get('email')):
                return f"{clean_response}\n\nThe email address provided doesn't seem to be valid. Please provide a valid email address."
            if not details.get('date'):
                missing.append("date")
            if not details.get('time'):
                missing.append("time")
                
            if missing:
                return f"{clean_response}\n\nI still need your {', '.join(missing)} to book the appointment."
            
            # Check if appointment exists
            if check_appointment_exists(details['name'], details['email'], details['date'], details['time']):
                return f"You already have an appointment on {details['date']} at {details['time']}. Would you like to book a different time?"
            
            # Book the appointment
            purpose = details.get('purpose', 'General appointment')
            add_appointment(details['name'], details['email'], details['date'], details['time'], purpose)
            st.session_state['current_name'] = details['name']
            st.session_state['current_email'] = details['email']
            
            return f"{clean_response}\n\nAppointment confirmed for:\nName: {details['name']}\nEmail: {details['email']}\nDate: {details['date']}\nTime: {details['time']}\nPurpose: {purpose}"
        
        elif details.get('action') == 'retrieve':
            # Get name and email for retrieval
            name = details.get('name') or st.session_state.get('current_name', '')
            email = details.get('email') or st.session_state.get('current_email', '')
            date = details.get('date')
            
            # Check if we have at least name or email
            if not name and not email:
                return f"{clean_response}\n\nPlease provide your name or email so I can check your appointments."
            
            # Update session state
            if name:
                st.session_state['current_name'] = name
            if email:
                st.session_state['current_email'] = email
            
            # Get appointments
            appointments = get_appointments(name, email, date)
            
            if not appointments:
                if date:
                    return f"{clean_response}\n\nYou don't have any appointments on {date}."
                else:
                    return f"{clean_response}\n\nYou don't have any appointments scheduled."
            
            response = f"{clean_response}\n\nHere are your appointments:\n\n"
            for appt in appointments:
                # Handle old database format (without email) or new format (with email)
                if len(appt) >= 6:  # New format with email
                    response += f"Name: {appt[1]}, Email: {appt[2]}, Date: {appt[3]}, Time: {appt[4]}, Purpose: {appt[5]}\n"
                else:  # Old format without email
                    response += f"Name: {appt[1]}, Date: {appt[2]}, Time: {appt[3]}, Purpose: {appt[4]}\n"
            
            return response
        
        # Default response if no action is determined but details were found
        return clean_response
    
    except Exception as e:
        return f"Error processing message: {str(e)}\n\nPlease try again or restart the application."

# Initialize the app
def main():
    st.set_page_config(page_title="AI Appointment Booking Agent", page_icon="📅")
    
    if 'messages' not in st.session_state:
        st.session_state['messages'] = [{"role": "assistant", "content": "Hello! I'm your AI appointment booking assistant. How can I help you today? I can help you book appointments or check your existing ones. Please provide your name and email when booking."}]
    
    if 'current_name' not in st.session_state:
        st.session_state['current_name'] = None
        
    if 'current_email' not in st.session_state:
        st.session_state['current_email'] = None
        
    if 'llm_chain' not in st.session_state:
        try:
            # Initialize LLM chain and store in session state
            st.session_state['llm_chain'] = setup_llm()
        except Exception as e:
            st.error(f"Error initializing LLM: {str(e)}")
            st.error("Please make sure you have set the GOOGLE_API_KEY environment variable or added it to Streamlit secrets.")
            st.stop()
    
    st.title("AI Appointment Booking Agent")
    
    # Initialize database
    try:
        init_db()
    except Exception as e:
        st.error(f"Database initialization error: {str(e)}")
        if os.path.exists('appointments.db'):
            st.error("The database file exists but there might be a schema issue. You may need to delete the file and restart.")
    
    # Display chat messages
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.write(message["content"])
    
    # User input
    user_input = st.chat_input("Type your message here...")
    
    if user_input:
        # Add user message to chat history
        st.session_state.messages.append({"role": "user", "content": user_input})
        with st.chat_message("user"):
            st.write(user_input)
        
        # Generate response using the LLM chain
        response = process_message(user_input, st.session_state['llm_chain'])
        
        # Add assistant response to chat history
        st.session_state.messages.append({"role": "assistant", "content": response})
        with st.chat_message("assistant"):
            st.write(response)

if __name__ == "__main__":
    main()